In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [27]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text

Get Toronto postal code data from wikipedia with the url above

In [28]:
soup = BeautifulSoup(html_data,'html5lib')

Using the BeautifulSoup library to perform web scraping on the specified url page

In [29]:
raw = []
for row in soup.find('table').find_all('td'):
  raw_row = {}
  if row.find('span').text == 'Not assigned':
    pass

  else:    
    var = row.find('span').text.split('(')

    raw_row['Postal Code'] = row.find('b').text
    raw_row['Borough'] = var[0]
    raw_row['Neighborhood'] = var[1]

    raw.append(raw_row)

It can be seen that the data you want to get is in the table, in the table it can be seen that in the span group there are borough and neighborhood, while for the Postal Code it is in the first 'b' group.

In [30]:
df = pd.DataFrame(raw)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods)
1,M4A,North York,Victoria Village)
2,M5A,Downtown Toronto,Regent Park / Harbourfront)
3,M6A,North York,Lawrence Manor / Lawrence Heights)
4,M7A,Queen's Park,Ontario Provincial Government)


In [31]:
df['Neighborhood'] = df['Neighborhood'].str.replace(r'\)$|\s$','').str.replace(r'\)',' ').str.replace(r'\s/',', ')
df['Borough'] = df['Borough'].replace({
    'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
    'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
    'EtobicokeNorthwest':'Etobicoke Northwest',
    'East YorkEast Toronto':'East York/East Toronto',
    'MississaugaCanada Post Gateway Processing Centre':'Mississauga'
    })

Neighborhood features are still very dirty with ')', '/', ''. Therefore, it is cleaned and tidied using replace and regex.
For Borough, there are some data that are not suitable, such as PO Boxes and others, so they need to be filtered and replaced with the appropriate data 

In [32]:
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, ..."


In [33]:
df.shape

(103, 3)

---


In [20]:
!wget -O GeoSpatial_Dataset.csv https://cocl.us/Geospatial_data

--2021-04-19 12:24:45--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 52.116.127.226, 52.116.127.228
Connecting to cocl.us (cocl.us)|52.116.127.226|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-04-19 12:24:46--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-04-19 12:24:46--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

Download the dataset regarding the latitude and longitude of the Toronto postal code. Because if you use the geocoder package it takes a very long time.

In [21]:
df_geo = pd.read_csv('GeoSpatial_Dataset.csv')

In [39]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
df_geo.shape

(103, 3)

It can be seen that the geospatial dataset has the same amount of data as the previous data, namely the postal code data in Toronto regarding the borough and neighborhood.

In [37]:
df_merge = df.merge(df_geo, how='inner', on='Postal Code')

The dataset is combined with a key value in the form of a Postal code. or if you are familiar with SQL then the term uses an inner join with a key in the form of a Postal Code

In [38]:
df_merge

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, ...",43.636258,-79.498509


In [42]:
df_merge.shape

(103, 5)